In [1]:
# import the necessary packages
from imutils.object_detection import non_max_suppression
import numpy as np
import time
import cv2
import skimage.io as io
import matplotlib.pyplot as plt
import os
import pytesseract
print(np.__version__)

1.14.3


In [2]:
def decode_predictions(scores, geometry):

    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []


    for y in range(0, numRows):

        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]


        for x in range(0, numCols):

            if scoresData[x] < min_confidence:
                continue


            (offsetX, offsetY) = (x * 4.0, y * 4.0)

            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]


            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)


            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])


    return (rects, confidences)

In [ ]:
# construct the argument parser and parse the arguments
image = cv2.imread("C:\\Users\\ckunwar\\Test_Images\\licence_plate1\\09.jpg")
east = 'frozen_east_text_detection.pb'
min_confidence = 0.1
width = 320
height = 320
padding = 0.0
# load the input image and grab the image dimensions

orig = image.copy()
(origH, origW) = image.shape[:2]

# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (width, height)
rW = origW / float(newW)
rH = origH / float(newH)

# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))

(H, W) = image.shape[:2]

# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = ["feature_fusion/Conv_7/Sigmoid","feature_fusion/concat_3"]

# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(east)

# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),(123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

# decode the predictions, then  apply non-maxima suppression to
# suppress weak, overlapping bounding boxes
(rects, confidences) = decode_predictions(scores, geometry)
boxes = non_max_suppression(np.array(rects), probs=confidences)

# initialize the list of results
results = []

# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
    # scale the bounding box coordinates based on the respective
    # ratios
    startX = int(startX * rW)
    startY = int(startY * rH)
    endX = int(endX * rW)
    endY = int(endY * rH)

    # in order to obtain a better OCR of the text we can potentially
    # apply a bit of padding surrounding the bounding box -- here we
    # are computing the deltas in both the x and y directions
    dX = int((endX - startX) * padding)
    dY = int((endY - startY) * padding)

    # apply padding to each side of the bounding box, respectively
    startX = max(0, startX - dX)
    startY = max(0, startY - dY)
    endX = min(origW, endX + (dX * 2))
    endY = min(origH, endY + (dY * 2))

    # extract the actual padded ROI
    roi = orig[startY:endY, startX:endX]

    config = ("-l eng --oem 3 --psm 11")
    text = pytesseract.image_to_string(roi,config=config)

    # add the bounding box coordinates and OCR'd text to the list
    # of results
    results.append(((startX, startY, endX, endY), text))

# sort the results bounding box coordinates from top to bottom
results = sorted(results, key=lambda r:r[0][1])
#print("results=",results)

output = orig.copy()
# loop over the results
for ((startX, startY, endX, endY), text) in results:
    # display the text OCR'd by Tesseract
    print("OCR TEXT")
    print("========")
    print("{}\n".format(text))
    
    # strip out non-ASCII text so we can draw the text on the image
    # using OpenCV, then draw the text and a bounding box surrounding
    # the text region of the input image
    text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
    
    cv2.rectangle(output, (startX, startY), (endX, endY),(0, 255, 0), 2)
    cv2.putText(output, text, (startX, startY - 5),cv2.FONT_HERSHEY_TRIPLEX, 0.8, (0, 0, 255), 2)

# show the output image
cv2.imwrite('C:\\Users\\ckunwar\\Test_Images\\licence_plate1_test\\test09.jpg', output)
cv2.imshow("Text Detection", output)
cv2.waitKey(0)

[INFO] loading EAST text detector...
OCR TEXT
AA-518-AB

